In [8]:
from openai import OpenAI
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle

In [9]:
# check if the openai api key is set in the environment
assert 'OPENAI_API_KEY' in os.environ.keys()
client = OpenAI()

In [10]:
# batched queries to chatGPT moderation API
def modAPI(model, text=["Sample text goes here.", "Sample text again goes here."]):
    response = client.moderations.create(input=text, model=model)
    outputs = response.results
    moderation_results = []
    for output in outputs:
        flagged = output.flagged
        categories_flagged = output.categories
        scores = output.category_scores
        moderation_results += [(flagged, categories_flagged, scores)]
    return moderation_results

In [11]:
def modRun(df, model='text-moderation-stable'):
    moderations = []
    for chunk in tqdm(chunker(df,10)):
        moderations += modAPI(model=model, text=chunk['text'].tolist())
    return moderations

In [19]:
data = pd.read_csv('./data/ideology_annotated.csv')
del data['Unnamed: 0']

In [13]:
def postprocessing(df, moderations):
    df['moderation'] = moderations
    df[['flagged_run', 'categories', 'scores']] = pd.DataFrame(df['moderation'].tolist(), index=df.index)
    df['categories'] = df['categories'].apply(lambda x: dict(x))
    df['scores'] = df['scores'].apply(lambda x: dict(x))
    return df

In [14]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [20]:
moderations = modRun(df=data[:10], model='text-moderation-stable')
outputs = postprocessing(data[:10], moderations)
print(outputs)

1it [00:00,  1.60it/s]

                                                text  label_id      subreddit  \
0  > On one hand fuck Russia, on the other hand I...         2      Socialism   
1  This could actually be a mind numbing amount o...         2  conservatives   
2  What if that self determination leads to fasci...         2      Socialism   
3  We need Benson to hold the seat.  Donated and ...         0      Democrats   
4  I do not like gerrymandering no matter who is ...         2      Democrats   
5  That is huge, size wise. Read a bit of chapter...         0      Socialism   
6  That's what happens when the US and UK empires...         2      Socialism   
7  Pleading the Fifth can be used against people ...         2      Democrats   
8  >”War is Peace\n\n>Freedom is Slavery \n\n>Ign...         2      Socialism   
9                     The shooter wasn't republican.         1      Democrats   

      label                                         moderation  flagged_run  \
0  balanced  (True, Categorie


/tmp/ipykernel_57737/2379791463.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['moderation'] = moderations
/tmp/ipykernel_57737/2379791463.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['flagged_run', 'categories', 'scores']] = pd.DataFrame(df['moderation'].tolist(), index=df.index)
/tmp/ipykernel_57737/2379791463.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 